This is an implementation based on the [too-many-cook](https://arxiv.org/pdf/2003.11778.pdf) workshop paper.

In [9]:
from google.colab import drive
drive.mount("/content/drive")
# %cd /content/drive/Shareddrives/Duong-LongWarwick/ROM
# !python -m atari_py.import_roms .
%cd '/content/drive/Shareddrives/Duong-KhoiBKHN/PantheonRL-Environment'
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/Duong-KhoiBKHN/PantheonRL-Environment
manycook_repo			overcooked_ai_py
OvercookedAI-Environment.ipynb	PantheonRL-MockEnvironment.ipynb


In [5]:
class manycook_arg:
    def __init__(self):
        # Environment
        self.level = 'open-divider_salad' 
        self.num_agents = 2 
        self.max_num_timesteps = 100
        self.max_num_subtasks = 14
        self.seed = 1
        self.with_image_obs = False
        self.beta = 1.3

        # Navigation Planner
        self.alpha = 0.01
        self.tau = 2
        self.cap = 75
        self.main_cap = 100

        # Visualizations 
        self.play = True
        self.record = False
        
        # Models 
        self.model1 = None 
        self.model2 = None 
        self.model3 = None 
        self.model4 = None



In [10]:
# from environment import OvercookedEnvironment
# from gym_cooking.envs import OvercookedEnvironment
from manycook_repo.recipe_planner.recipe import *
from manycook_repo.utils.world import World
from manycook_repo.utils.agent import RealAgent, SimAgent, COLORS
from manycook_repo.utils.core import *
from manycook_repo.misc.game.gameplay import GamePlay
from manycook_repo.misc.metrics.metrics_bag import Bag

import numpy as np
import random
import argparse
from collections import namedtuple

import gym



def fix_seed(seed):
    np.random.seed(seed)
    random.seed(seed)

def initialize_agents(arglist):
    real_agents = []

    with open('utils/levels/{}.txt'.format(arglist.level), 'r') as f:
        phase = 1
        recipes = []
        for line in f:
            line = line.strip('\n')
            if line == '':
                phase += 1

            # phase 2: read in recipe list
            elif phase == 2:
                recipes.append(globals()[line]())

            # phase 3: read in agent locations (up to num_agents)
            elif phase == 3:
                if len(real_agents) < arglist.num_agents:
                    loc = line.split(' ')
                    real_agent = RealAgent(
                            arglist=arglist,
                            name='agent-'+str(len(real_agents)+1),
                            id_color=COLORS[len(real_agents)],
                            recipes=recipes)
                    real_agents.append(real_agent)

    return real_agents

arglist = manycook_arg()
"""The main loop for running experiments."""
print("Initializing environment and agents.")
env = gym.envs.make("gym_cooking:overcookedEnv-v0", arglist=arglist)
obs = env.reset()
# game = GameVisualize(env)
real_agents = initialize_agents(arglist=arglist)

# Info bag for saving pkl files
bag = Bag(arglist=arglist, filename=env.filename)
bag.set_recipe(recipe_subtasks=env.all_subtasks)

while not env.done():
    action_dict = {}

    for agent in real_agents:
        action = agent.select_action(obs=obs)
        action_dict[agent.name] = action

    obs, reward, done, info = env.step(action_dict=action_dict)

    # Agents
    for agent in real_agents:
        agent.refresh_subtasks(world=env.world)

    # Saving info
    bag.add_status(cur_time=info['t'], real_agents=real_agents)


# Saving final information before saving pkl file
bag.set_collisions(collisions=env.collisions)
bag.set_termination(termination_info=env.termination_info,
        successful=env.successful)


ModuleNotFoundError: ignored